In [1]:
from deep_sort_realtime.deepsort_tracker import DeepSort
import tensorflow as tf
import cv2

import numpy as np

In [2]:
# 모델 불러오기
model = tf.saved_model.load('./COCO-SSD_Mobilenet_V2')

In [3]:
# 2. 로컬 영상 파일 열기
video_path = './video/1-1_608-C05.mp4'
cap = cv2.VideoCapture(video_path)

In [5]:
# 3. 프레임별 처리
tracker = DeepSort(max_age=30, nn_budget=70)  # DeepSORT 초기화

object_tracks = {}  # 객체 위치 저장용 딕셔너리
TARGET_CLASS = 1
count_num = 0

frame_count = 0
skip_frames = 5  # 5프레임마다 한 번씩 처리

print("영상시작")

# 첫 번째 프레임에서 프레임 크기 초기화
ret, frame = cap.read()
if not ret:  # 첫 번째 프레임 읽기 실패 시 프로그램 종료
    print("비디오 파일을 읽을 수 없습니다.")
    cap.release()
    exit()

frame_height, frame_width = frame.shape[:2]
mid_x = frame_width // 2  # 화면 중간 x좌표 계산

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % skip_frames != 0:  # 프레임 스킵
        continue
    
    # OpenCV BGR -> RGB 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(rgb_frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # 객체 탐지 실행
    detections = model(input_tensor)
    detection_boxes = detections['detection_boxes'][0].numpy()
    detection_classes = detections['detection_classes'][0].numpy()
    detection_scores = detections['detection_scores'][0].numpy()

    # 현재 프레임에서 탐지된 객체 정보 저장
    current_frame_tracks = {}

    # 탐지 결과 표시
    dets = []

    for i in range(len(detection_scores)):
        if detection_scores[i] > 0.5 and detection_classes[i] == TARGET_CLASS:  # 객체 분류에 대한 신뢰도 임계값 0.5 이상으로 설정

            # 경계 상자(지된 객체의 위치를 지정하는 사각형 영역) 좌표(실제 크기) 계산
            # box = detection_boxes[i] * [frame.shape[0], frame.shape[1], frame.shape[0], frame.shape[1]]
            # (ymin, xmin, ymax, xmax) = box.astype('int')

            # 객체의 중심점 계산
            # center_x = int((xmin + xmax) / 2)
            # center_y = int((ymin + ymax) / 2)

            # 임시로 객체 ID를 인덱스 `i`로 사용 (더 나은 추적 알고리즘 필요 -> 차후 deep_sort로
            # object_id = i
            # current_frame_tracks[object_id] = (center_x, center_y)

            # # 이전 위치와 비교하여 이동 백터 계산
            # if object_id in object_tracks:
            #     prev_center = object_tracks[object_id]
            #     prev_center_x, prev_center_y = object_tracks[object_id]
            #     current_center = (center_x, center_y)
            #     direction = (current_center[0] - prev_center[0], current_center[1] - prev_center[1])

                # # 이동 거리가 너무 크면 노이즈로 간주
                # if abs(center_x - prev_center_x) > frame_width * 0.8:
                #     continue

                # # 왼쪽 -> 오른쪽으로 이동
                # if prev_center_x < mid_x and center_x >= mid_x:
                #     count_num += 1
                #     print(f"객체 {object_id}가 왼쪽에서 오른쪽으로 이동. 현재 인원수: {count_num}")
                
                # # 오른쪽 -> 왼쪽으로 이동
                # elif prev_center_x > mid_x and center_x <= mid_x:
                #     count_num -= 1
                #     print(f"객체 {object_id}가 오른쪽에서 왼쪽으로 이동. 현재 인원수: {count_num}")

                # 이동 방향 화살표
                # cv2.arrowedLine(frame, prev_center, current_center, (0, 0, 255), 2, tipLength=0.3)

            # # 현재 위치를 이전 위치로 업데이트
            # object_tracks[object_id] = (center_x, center_y)

        # # 경계 상자 및 중심점 시각화
        # cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        # cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
    
    # # 객체 이동 방향 업데이트
    # object_tracks = current_frame_tracks

            box = detection_boxes[i] * [frame.shape[0], frame.shape[1], frame.shape[0], frame.shape[1]]
            ymin, xmin, ymax, xmax = box.astype('int')
            dets.append([float(xmin), float(ymin), float(xmax), float(ymax), float(detection_scores[i])])  # 형식 변환

            # DeepSORT 추적 실행
            if dets:  # dets가 비어 있지 않은 경우에만 호출
                print("Detections:", dets)  # 디버깅 출력
                tracks = tracker.update_tracks(dets, frame=frame)

            # 추적 결과 처리
            for track in tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue

                track_id = track.track_id  # 객체 ID
                l, t, r, b = track.to_ltrb()  # 추적된 경계 상자 좌표
                center_x = int((l + r) / 2)

                # 왼쪽 -> 오른쪽으로 이동
                if center_x < mid_x and center_x >= mid_x:
                    count_num += 1
                    print(f"객체 {track_id}가 왼쪽에서 오른쪽으로 이동. 현재 인원수: {count_num}")

                # 오른쪽 -> 왼쪽으로 이동
                elif center_x > mid_x and center_x <= mid_x:
                    count_num -= 1
                    print(f"객체 {track_id}가 오른쪽에서 왼쪽으로 이동. 현재 인원수: {count_num}")

                # 경계 상자 및 ID 시각화
                cv2.rectangle(frame, (int(l), int(t)), (int(r), int(b)), (0, 255, 0), 2)
                cv2.putText(frame, f'ID: {track_id}', (int(l), int(t) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    
    # 영상 중간 부분에 세로선 그리기
    cv2.line(frame, (mid_x, 0), (mid_x, frame_height), (255, 0, 0), 2)  # (255, 0, 0)은 파란색, 두께는 2

    # 프레임 보여주기
    cv2.imshow('Detection with Direction', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


영상시작
Detections: [[593.0, 256.0, 772.0, 690.0, 0.577471137046814]]


TypeError: object of type 'float' has no len()